In [1]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random
import numpy as np

In [2]:
def open_json(filename):
    with open(filename, "r", encoding="utf-8") as read_file:
        data = json.load(read_file)
    return data

In [94]:
class Person:
    def __init__(self, dictionary):
        self.id = dictionary['id']
        self.name = dictionary['name']
        self.gender = dictionary['gender']
        if 'pids' in dictionary.keys():
            self.pids = dictionary['pids']
        if 'mid' in dictionary.keys():
            self.mid = dictionary['mid']
        if 'fid' in dictionary.keys():
            self.fid = dictionary['fid']
        self.cids = []
        self.rank = -1

    def __repr__(self):
        result = str(self.id) + ', ' + self.name + ', ' + self.gender
        if hasattr(self, 'pids'):
            result += ', ' + str(self.pids)
        if hasattr(self, 'mid'):
            result += ', ' + str(self.mid)
        if hasattr(self, 'fid'):
            result += ', ' + str(self.fid)
        if len(self.cids) != 0:
            result += ', ' + str(self.cids)
        result += ', ' + str(self.rank)
        return '{' + result + '}'

In [105]:
def get_all_rank(tree, rank):
    result = []
    for id in tree:
        if tree[id].rank == str(rank):
            result.append(tree[id])
    return result

In [119]:
class FamilyTree:
    def __init__(self, tree_list):
        self.tree = {}
        if type(tree_list) is list:
            for node in tree_list:
                if type(node) is dict:
                    person = Person(node)
                    self.tree.update({person.id: person})
                else:
                    print("List must contains dict!")
        else:
            print("Tree must be list!")
        self.__add_children()
        self.__add_rank()

    def __add_children(self):
        result = self.tree
        for i in self.tree:
            if hasattr(self.tree[i], "mid"):
                result[self.tree[i].mid].cids.append(self.tree[i].id)
            if hasattr(self.tree[i], "fid"):
                result[self.tree[i].fid].cids.append(self.tree[i].id)
        self.tree = result

    def __add_rank(self):
        result = self.tree
        def add_rank_node(node, rank):
            if node.rank == -1:
                node.rank = str(rank)
                if hasattr(node, 'pids'):
                    for n in node.pids:
                        add_rank_node(result[n], rank)
            else:
                rank+=1
                if len(node.cids) != 0:
                    for n in node.cids:
                        add_rank_node(result[n], rank)
        add_rank_node(result[1], 0)
        self.tree = result

In [120]:
family_tree = FamilyTree(open_json('data/tree3.json'))
family_tree.tree

{1: {1, Михаил Федорович, male, [2], [3], 0},
 2: {2, Евдокия Лукьяновна Стрешнева, female, [1], [3], 0},
 3: {3, Алексей Михайлович, male, [4, 5], 1, 2, [6, 7, 8, 18], 1},
 4: {4, Мария Ильинична Милославская, female, [3], [6, 7, 8], 1},
 5: {5, Наталья Кирилловна Нарышкина, female, [3], [18], 1},
 6: {6, Федор Алексеевич, male, [9], 3, 4, 2},
 7: {7, Софья Алексеевна, female, 3, 4, 2},
 8: {8, Иоанн V Алексеевич, male, [10], 3, 4, [11, 12], 2},
 9: {9, Марфа Матвеевна Апраксина, female, [6], 2},
 10: {10, Прасковья Федоровна Салтыкова, female, [8], [11, 12], 2},
 11: {11, Екатерина Иоановна, female, [13], 8, 10, [15], 3},
 12: {12, Анна Иоановна, female, [14], 8, 10, 3},
 13: {13, Карл Леопольд Мекленбург-Шверенский, male, [11], [15], 3},
 14: {14, Фридрих Вильгельм Курляндский, male, [12], 3},
 15: {15, Анна Леопольдовна, female, [16], 13, 11, [17], 4},
 16: {16, Антон Ульрих Брауншвейг-Вольфенбюттельский, male, [15], [17], 4},
 17: {17, Иоанн VI Антонович, male, 15, 16, 5},
 18: {1

In [130]:
len(get_all_rank(family_tree.tree, 9))

2

In [50]:
test = Person({"id": 17, "mid": 15, "fid": 16, "name": "Иоанн VI Антонович", "gender": "male"})
print(test)

{17, Иоанн VI Антонович, male, 15, 16}


In [51]:
test.cids.append(1)

In [52]:
test.cids

[1]